In [1]:
import pandas as pd
import numpy as np
import web3
import json
import requests
from addict import Addict as ad
from collections import defaultdict
import concurrent.futures
from eth_abi import decode

import gspread

### Sepolia

In [2]:
ep = 'http://46.4.113.167:13888'
w3 = web3.Web3(web3.HTTPProvider(ep))
w3.is_connected()

True

In [3]:
# ABI_ENDPOINT = 'https://api.etherscan.io/api?module=contract&action=getabi&address='

ABI_ENDPOINT = 'https://api-sepolia.etherscan.io/api?module=contract&action=getabi&address='
AGENT_ADDRESS = '0xbdE2Aed54521000DC033B67FB522034e0F93A7e5' #'0xf4583fc017D82c3462944A5d7E7aD380e5bfAD74'
AGENT_IMPL = '0x4deA5Ec11e1EB6FF7fa62EED2fA72a1aE2934E89' #'0xb455b75EbFfA79902cAd95B2847318e1E1AaAdb9'

abi = json.loads(requests.get(ABI_ENDPOINT+AGENT_IMPL).json()['result'])
agent = w3.eth.contract(address=AGENT_ADDRESS, abi = abi)

In [4]:
def keeperList(window_days):
    lbn = w3.eth.get_block_number()
    days = range(1, window_days+1)
    executes = ad(defaultdict())
    for d in days:
        executesFilter = w3.eth.filter({"address": AGENT_ADDRESS,
                    "topics": ['0x59200007defbdd8d2c9e53dd05f81091fe977b424f38f2562df9ee8725d58469', None, None,],
                    'fromBlock': lbn-int(d*24*3600/12),
                    'toBlock': lbn-int((d-1)*24*3600/12)
                    })
        
        executesEvents = executesFilter.get_all_entries()
#       while True:
#             try:
#                 executesEvents = executesFilter.get_all_entries()
#                 if len(executesEvents) > 0:
#                     break
#             except:
#                 pass
        
        executesUnique = set([evt.topics[3] for evt in executesEvents])
        ade = ad({int(keeper.hex(), base = 16): sum([1 if event.topics[3] == keeper else 0 for event in executesEvents]) for keeper in executesUnique})
        for k in ade:
            #executes[k]+=round(ade[k]/d, 2)
            executes[k]+=ade[k]
    
    empty_keepers = set(agent.functions.getActiveKeepers().call()).difference(set(executes.keys()))
    for i in empty_keepers:
        executes[i] = 0
    
    #keeperSet = set(executes.keys())
    # for k in keeperSet:
    #     print(agent.functions.getKeeper(k).call()[0])
    return dict(sorted(executes.items(), key=lambda item: item[1], reverse=True))

In [5]:
def keeperSlashes(window_days):
    lbn = w3.eth.get_block_number()
    days = range(1, window_days+1)
    executes = ad(defaultdict())
    for d in days:
        slashesFilter = w3.eth.filter({"address": AGENT_ADDRESS,
                    "topics": ['0xf23cbf84c67ef352ebbbd099d226197ad56519c307692b0305363626c38054dd', None, None,],
                    'fromBlock': lbn-int(d*24*3600/12),
                    'toBlock': lbn-int((d-1)*24*3600/12)
                    })
        
        slashesEvents = slashesFilter.get_all_entries()
#       while True:
#             try:
#                 slashesEvents = slashesFilter.get_all_entries()
#                 if len(slashesEvents) > 0:
#                     break
#             except:
#                 pass
        
        slashesUnique = set([evt.topics[2] for evt in slashesEvents])
        ade = ad({int(keeper.hex(), base = 16): sum([1 if event.topics[2] == keeper else 0 for event in slashesEvents]) for keeper in slashesUnique})
        for k in ade:
            executes[k]+=ade[k]
    
    empty_keepers = set(agent.functions.getActiveKeepers().call()).difference(set(executes.keys()))
    for i in empty_keepers:
        executes[i] = 0
    
    #keeperSet = set(executes.keys())
    # for k in keeperSet:
    #     print(agent.functions.getKeeper(k).call()[0])
    return dict(sorted(executes.items(), key=lambda item: item[1], reverse=True))

In [6]:
def keeperActivations(window_days):
    lbn = w3.eth.get_block_number()
    days = range(1, window_days+1)
    executes = ad(defaultdict())
    for d in days:
        activationsFilter = w3.eth.filter({"address": AGENT_ADDRESS,
                    "topics": ['0x6f9aad08d1e095cdf28933bc980cde7cae350adab7abd43a6f45b4711f91554a', None, None,],
                    'fromBlock': lbn-int(d*24*3600/12),
                    'toBlock': lbn-int((d-1)*24*3600/12)
                    })
        activationsEvents = activationsFilter.get_all_entries()
        activationsUnique = set([evt.topics[1] for evt in activationsEvents])
        ade = ad({int(keeper.hex(), base = 16): sum([1 if event.topics[1] == keeper else 0 for event in activationsEvents]) for keeper in activationsUnique})
        for k in ade:
            executes[k]+=ade[k]
    
    empty_keepers = set(agent.functions.getActiveKeepers().call()).difference(set(executes.keys()))
    for i in empty_keepers:
        executes[i] = 0
    
    #keeperSet = set(executes.keys())
    # for k in keeperSet:
    #     print(agent.functions.getKeeper(k).call()[0])
    return dict(sorted(executes.items(), key=lambda item: item[1], reverse=True))

In [7]:
### active keepers ###

# keeperInfo = keeperList(30)
# keepers = pd.DataFrame(list(keeperInfo.items()), columns=['KeeperId', 'Executions'])
# keepers

### new jobs ###

# lbn = w3.eth.get_block_number()
# days = range(1, 30)
# new_jobs = set()
# for d in days:
#     jobRegistrationsFilter = w3.eth.filter({"address": AGENT_ADDRESS,
#                 "topics": ['0xb9b4668f2492830c54dc2fa7c8e28d9184882690fbedea4abeb03f016663d212', None, None,],
#                 'fromBlock': lbn-int(d*24*3600/12),
#                 'toBlock': lbn-int((d-1)*24*3600/12)
#                 })
#     jobRegistrationsEvents = jobRegistrationsFilter.get_all_entries()
#     for evt in jobRegistrationsEvents:
#         new_jobs.add(evt.topics[1])
# print(len(new_jobs))

### jobs executions ###

# keepers['Executions'].sum()

In [8]:
keeperInfo = keeperList(7)
keepers = pd.DataFrame(list(keeperInfo.items()), columns=['KeeperId', 'Executions'])
keepers

,KeeperId,Executions
0,92,951
1,76,948
2,33,931
3,74,930
4,97,929
5,82,919
6,94,913
7,7,911
8,96,910
9,8,908


In [9]:
keeperInfo2 = keeperSlashes(7)
keepers2 = pd.DataFrame(list(keeperInfo2.items()), columns=['KeeperId', 'Slashed'])
keepers2

,KeeperId,Slashed
0,78,78
1,13,57
2,95,53
3,11,28
4,93,27
5,67,20
6,89,19
7,94,14
8,33,11
9,96,10


In [10]:
ep = 'https://eth-sepolia.g.alchemy.com/v2/qgqQ7cYq3iPWHa22I2RNxE-LxJCJsKae'
w3 = web3.Web3(web3.HTTPProvider(ep))
w3.is_connected()

# ABI_ENDPOINT = 'https://api.etherscan.io/api?module=contract&action=getabi&address='

ABI_ENDPOINT = 'https://api-sepolia.etherscan.io/api?module=contract&action=getabi&address='
AGENT_ADDRESS = '0xbdE2Aed54521000DC033B67FB522034e0F93A7e5'
AGENT_IMPL = '0x4deA5Ec11e1EB6FF7fa62EED2fA72a1aE2934E89'

abi = json.loads(requests.get(ABI_ENDPOINT+AGENT_IMPL).json()['result'])
agent = w3.eth.contract(address=AGENT_ADDRESS, abi = abi)

In [11]:
keeperInfo3 = keeperActivations(7)
keepers3 = pd.DataFrame(list(keeperInfo3.items()), columns=['KeeperId', 'Activations'])
keepers3

,KeeperId,Activations
0,78,1
1,7,0
2,8,0
3,74,0
4,11,0
5,76,0
6,13,0
7,77,0
8,82,0
9,89,0


In [12]:
keepers['Admin'] = '-'
for i in range(len(keepers)):
    keepers['Admin'][i] = agent.functions.getKeeper(int(keepers['KeeperId'][i])).call()[0]

# keepers = keepers.loc[~keepers['Admin'].isin([
#     '0xdd5e98Cc27d451EF7dB2FD14c85C65630C10863B',
#     '0xc138f59a32F67FB2E835f81413E1102a134cDd01',
#     '0xC4F8115DC255C3cA003Ee69476533db142469401',
#     '0xDf4f9355572669ddbD2938AC0d7613F965e327B5',
#     '0xCce0bca1365a02e5770390165E64d1F59238D92e',
#     '0xa6DF7c99eeE3874fed703470C50EAfE0Cb56d19c',
#     '0x8851a74FE9247D6DB6A85b52e76D55078E0726A2',
#     '0xe95a2A5A39BE9E5F4C9758FE692Cc6dbE2D182E7'
#     ])].reset_index(drop=True)
keepers

C:\Users\Barca\AppData\Local\Temp\ipykernel_1336\457363654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keepers['Admin'][i] = agent.functions.getKeeper(int(keepers['KeeperId'][i])).call()[0]


,KeeperId,Executions,Admin
0,92,951,0x95e2e66eC1cCd659658Ea7a4484d8CB64b23be37
1,76,948,0xE8dF771Be36Ee789dD6C505832B6c4826B455F51
2,33,931,0x161E5CD8627Ba64e30310894D5aa70F379e78E15
3,74,930,0x585D4C381B912E8FD43f9cf81c62dbB73f9b09da
4,97,929,0x402491915eE71A51ce6AAA022a2E0fb6aC9B9c9c
5,82,919,0xAeb0212df65373cF775832E8dA4BC9Ee2c81C31B
6,94,913,0x9e9883cfc302310F72C2c85d9829E6c7Ddcf9A28
7,7,911,0x8bbf3B292Ce0fc7bA7d624A6cb6AB0813D020597
8,96,910,0xe6b5A31d8bb53D2C769864aC137fe25F4989f1fd
9,8,908,0x54e62D52d3AE3C88718edd4Ec084972F66222528


In [13]:
new = keepers.merge(keepers2, how='left', on='KeeperId')
new.loc[new['Slashed'].isin([np.nan]), 'Slashed'] = 0
new

,KeeperId,Executions,Admin,Slashed
0,92,951,0x95e2e66eC1cCd659658Ea7a4484d8CB64b23be37,3
1,76,948,0xE8dF771Be36Ee789dD6C505832B6c4826B455F51,2
2,33,931,0x161E5CD8627Ba64e30310894D5aa70F379e78E15,11
3,74,930,0x585D4C381B912E8FD43f9cf81c62dbB73f9b09da,4
4,97,929,0x402491915eE71A51ce6AAA022a2E0fb6aC9B9c9c,0
5,82,919,0xAeb0212df65373cF775832E8dA4BC9Ee2c81C31B,3
6,94,913,0x9e9883cfc302310F72C2c85d9829E6c7Ddcf9A28,14
7,7,911,0x8bbf3B292Ce0fc7bA7d624A6cb6AB0813D020597,1
8,96,910,0xe6b5A31d8bb53D2C769864aC137fe25F4989f1fd,10
9,8,908,0x54e62D52d3AE3C88718edd4Ec084972F66222528,5


In [14]:
new2 = new.merge(keepers3, how='left', on='KeeperId')
new2.loc[new2['Activations'].isin([np.nan]), 'Activations'] = 0
new2

,KeeperId,Executions,Admin,Slashed,Activations
0,92,951,0x95e2e66eC1cCd659658Ea7a4484d8CB64b23be37,3,0.0
1,76,948,0xE8dF771Be36Ee789dD6C505832B6c4826B455F51,2,0.0
2,33,931,0x161E5CD8627Ba64e30310894D5aa70F379e78E15,11,0.0
3,74,930,0x585D4C381B912E8FD43f9cf81c62dbB73f9b09da,4,0.0
4,97,929,0x402491915eE71A51ce6AAA022a2E0fb6aC9B9c9c,0,0.0
5,82,919,0xAeb0212df65373cF775832E8dA4BC9Ee2c81C31B,3,0.0
6,94,913,0x9e9883cfc302310F72C2c85d9829E6c7Ddcf9A28,14,0.0
7,7,911,0x8bbf3B292Ce0fc7bA7d624A6cb6AB0813D020597,1,0.0
8,96,910,0xe6b5A31d8bb53D2C769864aC137fe25F4989f1fd,10,0.0
9,8,908,0x54e62D52d3AE3C88718edd4Ec084972F66222528,5,0.0


In [15]:
gc = gspread.service_account()
sheet = gc.open("Testers Leaderboard")

wks = sheet.worksheet("sepolia_17.06")
wks.update_title("sepolia_01.07")

# wks = sheet.add_worksheet(title="sepolia_", rows=200, cols=20)

wks.update([new2.columns.values.tolist()] + new2.values.tolist())

C:\Users\Barca\anaconda3\lib\site-packages\gspread\worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '1JTgOTRPzeJmmXTDCPRpAcPO7jscl-HF1uaT0S_AY2ag',
 'updatedRange': 'sepolia_01.07!A1:E19',
 'updatedRows': 19,
 'updatedColumns': 5,
 'updatedCells': 95}